In [6]:
# Dependencies:

#!pip install transformers ipywidgets IPython

# If running on Colab:
#!git clone https://github.com/sam-paech/antislop-sampler.git
#!mv antislop-sampler/src .
#!mv antislop-sampler/slop_phrase_prob_adjustments.json .

In [1]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from src.antislop_generate import generate_antislop, chat_antislop

# Enable efficient transfer for Hugging Face models
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = "1"

# Set the device to 'cuda' if available, else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the model name (replace with your preferred model)
model_name = "unsloth/Llama-3.2-1B-Instruct"
#model_name = "unsloth/Llama-3.2-3B-Instruct"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, trust_remote_code=True)
model.to(device)
print('Model loaded')

Model loaded


In [2]:
import json
if os.path.exists('slop_phrase_prob_adjustments.json'):
    with open('slop_phrase_prob_adjustments.json', 'r') as f:
        slop_phrase_prob_adjustments = dict(json.load(f)[:500])

In [3]:
prompt = "Write a story about Elara, the weaver of tapestries in future Technopolis. In the bustling city, a group of "

In [ ]:
# Chat generation with streaming
messages = [
    {"role": "user", "content": prompt}
]
tokens = []
text = ''
for token in chat_antislop(
    model=model,
    tokenizer=tokenizer,
    messages=messages,
    max_new_tokens=400,
    # Antislop sampling may be less reliable at low temperatures.
    #temperature=1,    
    #min_p=0.1,
    temperature=0.01,
    # The adjustment_strength param scales how strongly the probability adjustments are applied.
    # A value of 1 means the values in slop_phrase_prob_adjustments (or the defaults) are used unmodified.
    # Reasonable values are 0 (disabled) thru 100+ (effectively banning the list).
    adjustment_strength=100.0,
    # Optional: Provide a list of slop phrases and probability adjustments
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    enforce_json=False,
    antislop_enabled=False,
    ban_slop_first_tokens=True,
    streaming=True,
    stream_smoothing=True, # On by default; this will smooth out the stutters from backtracking.
):
    tokens.append(token)
    full_text = tokenizer.decode(tokens, skip_special_tokens=True)
    new_text = full_text[len(text):]
    text = full_text
    print(new_text, end='', flush=True)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In the heart of Technopolis, a city of towering skyscrapers and humming machinery, Elara's workshop stood out among the sea of identical, high-tech buildings. The sign above her door read "Moonwhisper's Tapestry Weavers," and the windows were filled with a kaleidoscope of colors, as if the very fabric of the city itself was woven into the tapestries.

Elara, a skilled weaver with hair as silver as the moon and eyes that shone like

In [11]:
# Chat generation without streaming
messages = [
    {"role": "user", "content": prompt}
]
generated_text = chat_antislop(
    model=model,
    tokenizer=tokenizer,
    messages=messages,
    max_length=400,
    temperature=1,
    min_p=0.1,
    adjustment_strength=100.0,
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    enforce_json=False,
    antislop_enabled=True,
    regex_bans=['(?i)not [^.!?]{3,60} but'], # if these regex expressions are matched, we backtrack, ban that continuation and retry
    streaming=False
)
print(tokenizer.decode(generated_text))

In the heart of Future City, a sprawling technological marvel, the air was alive with the hum of machinery and the chatter of pedestrians. The city's residents moved with purpose, their footsteps echoing off the sleek, silver skyscrapers that pierced the clouds. Among the throngs was a young woman named Elian, her dark hair tied back in a tight ponytail as she hurried down the main thoroughfare.

Elian was a weaver, one of the city's skilled artisans who brought beauty and magic to the fabric of its inhabitants. Her workshop, a cozy room tucked away in a quiet alley, was a haven of creativity and tranquility. As she worked, her fingers moved deftly, the shuttle of her loom weaving a tale of wonder and enchantment.

The weavers of Future City were renowned for their extraordinary talent, their creations imbued with the essence of the city's energy. Elian's own work was a fusion of traditional techniques and advanced machinery, allowing her to craft fabrics that shone like stars and puls

In [12]:
# generate without streaming
prompt_with_template = tokenizer.apply_chat_template(messages, tokenize=False)
generated_text = generate_antislop(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_length=300,
    temperature=1,
    min_p=0.1,
    adjustment_strength=100.0,
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    enforce_json=False,
    antislop_enabled=True,
    streaming=False
)        
print(tokenizer.decode(generated_text))

 urchins stumble upon her at the market square, and a young girl named Aria discovers her talent.

---

The sun beat down on the market square, casting a warm glow over the crowded stalls. Vendors hawked their wares – fresh produce, exotic spices, and handmade crafts – while the smell of roasting meats and baking bread filled the air. In the midst of it all, a lone weaver sat at her stall, her fingers moving deftly as she wove a new design onto a large piece of fabric.

The weaver, a young woman with a wild tangle of curly hair and a warm smile, worked with a quiet focus. Her fingers flew across the loom, the threads shimmering and glowing as she brought her vision to life. Aria, a young girl with a mop of curly brown hair and a curious gaze, wandered through the market, her eyes scanning the stalls for scraps of fabric to use in her own projects.

As she passed by the weaver's stall, Aria caught sight of the beautiful, swirling patterns that covered the fabric. She felt an instant con

In [13]:
# generate with streaming
prompt_with_template = tokenizer.apply_chat_template(messages, tokenize=False)
tokens = []
text = ""
for token in generate_antislop(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_length=300,
    temperature=1,
    min_p=0.1,
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    adjustment_strength=100.0,
    enforce_json=False,
    antislop_enabled=True,
    streaming=True
):
    tokens.append(token)
    full_text = tokenizer.decode(tokens, skip_special_tokens=True)
    new_text = full_text[len(text):]
    text = full_text
    print(new_text, end='', flush=True)

 5 people gather in a hidden alleyway, each with a mysterious device in their hand. They are about to participate in a secret event, where they will compete in a tournament of wit and strategy, but as they begin, they realize that each of them has been sent by a powerful corporation to spy on each other, and a mysterious figure lurks in the shadows.

The group consists of:

1.  **Aurora, a skilled hacker from the cybernetic gang, "The Codekeepers"**
2.  **Lysander, a master strategist from the espionage agency "The Shadowhand"**
3.  **Mira, a brilliant scientist from the research facility "The Nexus"**
4.  **Caspian, a charismatic con artist with ties to the underworld**
5.  **Nova, a rebellious young artist with a hidden past**

As they enter the tournament hall, they are greeted by the host, **Ryker**, a charismatic and suave corporate executive. Ryker explains the rules of the tournament, but as the competitors begin to mingle and exchange information, they start to realize that som